In [ ]:
! pip install datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# load the dataset and the tokenizers
data = load_dataset('magicsword/train-en-zh')['train']
print(data['text'][0])
tokenizer_en = AutoTokenizer.from_pretrained('bert-base-cased')
tokenizer_zh = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')
print("English vocab:")
print('length: ',len(tokenizer_en.vocab))
print('special tokens: ',tokenizer_en.all_special_tokens)
print('special tokens ids: ',tokenizer_en.all_special_ids)
print("Chinese vocab:")
print('length: ',len(tokenizer_zh.vocab))
print('special tokens: ',tokenizer_zh.all_special_tokens)
print('special tokens ids: ',tokenizer_zh.all_special_ids)

In [ ]:
def map_fn(example):
  return {'English':[t.split('|||')[0] for t in example['text']],
          'Chinese':[t.split('|||')[1] for t in example['text']]}

data = data.map(map_fn,batched=True)
data = data.remove_columns(['text'])
data['English'][:3],data['Chinese'][:3]

In [ ]:
def tokenize_fn_en(example):
  return {'tokenized_English':[tokenizer_en.tokenize(en,add_special_tokens=True) for en in example['English']]}
def tokenize_fn_zh(example):
  return {'tokenized_Chinese':[tokenizer_zh.tokenize(zh,add_special_tokens=True) for zh in example['Chinese']]}

data = data.map(tokenize_fn_en,batched=True)
data = data.map(tokenize_fn_zh,batched=True)
data = data.remove_columns(['English','Chinese'])
print(data['tokenized_English'][14])
print(data['tokenized_Chinese'][14])

In [ ]:
def tokenize_id_en(example):
  return {'tokenized_en_id':[tokenizer_en.convert_tokens_to_ids(en) for en in example['tokenized_English']]}
def tokenize_id_zh(example):
  return {'tokenized_zh_id':[tokenizer_zh.convert_tokens_to_ids(zh) for zh in example['tokenized_Chinese']]}

data = data.map(tokenize_id_en,batched=True)
data = data.map(tokenize_id_zh,batched=True)
print(data['tokenized_en_id'][14])
print(data['tokenized_zh_id'][14])

In [ ]:
print(data['tokenized_en_id'][15])
print(data['tokenized_zh_id'][15])

In [ ]:
print(data['tokenized_en_id'][2])
print(data['tokenized_zh_id'][2])

In [ ]:
tokenizer_en.decode(data['tokenized_en_id'][1]),tokenizer_zh.decode(data['tokenized_zh_id'][1])

In [ ]:
data.to_json('/content/drive/MyDrive/2024Spring/641NaturalLanguageProcessing/NewFolder/dataset.json')